In [6]:
import numpy as np
from numba import njit
import pickle
import time, sys

from sympy import Matrix

from scipy.optimize import curve_fit
from scipy.interpolate import splrep, splev

import matplotlib.pyplot as plt

In [27]:
n_orb = 3

def FockState(index):
    return np.array(list(np.binary_repr(index - 1, width=2*n_orb)), dtype=int)

def FockIndex(bin_):
    return int("".join(str(x) for x in bin_), 2) + 1

def FermiCop(flavor):
    def condition(index):
        return FockState(index)[flavor-1] == 0

    indices = [index for index in range(1, 4**n_orb + 1) if condition(index)]
    sum_matrix = np.zeros((4**n_orb, 4**n_orb))
    for index in indices:
        state = FockState(index)
        phase = (-1)**np.sum(state[:flavor])
        updated_state = np.copy(state)
        updated_state[flavor-1] = 1
        sum_matrix += phase * np.outer(np.eye(4**n_orb)[index-1], np.eye(4**n_orb)[FockIndex(updated_state)-1])
    return sum_matrix

cun = np.array([FermiCop(i) for i in range(1, n_orb+1)]).astype(np.csingle)
cdn = np.array([FermiCop(i) for i in range(n_orb+1, 2*n_orb+1)]).astype(np.csingle)
cn = np.array([FermiCop(i) for i in range(1, 2*n_orb+1)]).astype(np.csingle)


In [20]:
@njit
def nonintmat(kx, n_orb):
    A = np.exp(1j * kx / n_orb)
    mat = np.zeros((n_orb, n_orb), dtype=np.csingle)

    for i in range(n_orb):
        for j in range(n_orb):
            if i - j == 1:
                mat[i, j] += 1 / A
            if j - i == 1:
                mat[i, j] += A

    mat[0, n_orb - 1] += 1 / A
    mat[n_orb - 1, 0] += A

    return mat

@njit
def hamiltonian(nonint, mu, U):
    nonint_term = np.zeros((4**n_orb, 4**n_orb), dtype=np.csingle)
    U_term = np.zeros((4**n_orb, 4**n_orb), dtype=np.csingle)
    mu_term = np.zeros((4**n_orb, 4**n_orb), dtype=np.csingle)

    for i in range(n_orb):
        for j in range(n_orb):
            nonint_term += np.dot(cun[i].T, cun[j]) * nonint[i, j]
            nonint_term += np.dot(cdn[i].T, cdn[j]) * nonint[i, j]
        U_term += np.dot(np.dot(cun[i].T , cun[i]) ,np.dot( cdn[i].T ,cdn[i])) * U
    for i in range(2*n_orb):
        mu_term += -mu * (cn[i].T @ cn[i])


    # Total Hamiltonian
    H = nonint_term + mu_term + U_term
    return H+np.eye(4**n_orb, dtype=np.csingle)*U


@njit
def fullhamil(kx, t, mu, U):
    return hamiltonian(-t * nonintmat(kx, n_orb), mu, U)

In [21]:
H = fullhamil(0, 1, 5, 10)

In [29]:
@njit
def compute_N(s, n_orb):
    N = 0
    for i in range(n_orb):
        d = (s // (4**i)) % 4
        if d == 1 or d == 2:
            N += 1
        elif d == 3:
            N += 2
    return N

def generate_basis_indices(n_orb, desired_N):
    basis_indices = []
    for s in range(4**n_orb):
        if compute_N(s, n_orb) == desired_N:
            basis_indices.append(s)
    return np.array(basis_indices, dtype=np.int64)

# Assuming H is the full Hamiltonian obtained from fullhamil()
basis_indices = generate_basis_indices(n_orb, 2)
H_N = H[np.ix_(basis_indices, basis_indices)]